### R Package list

The following R packages are used in this notebook.

ggplot2()  
ggdendro()  
Rlibstree() (a Bioconductor package)   
reshape()  
scales()    

In [3]:
.libPaths()

[1] "/Users/veldhuis/Library/R/3.2/library"                         
[2] "/Library/Frameworks/R.framework/Versions/3.2/Resources/library"

In [ ]:
lib.dir = "/Library/Frameworks/R.framework/Versions/3.2/Resources/library"

#install.packages('ggplot2', lib=lib.dir)
#install.packages('ggdendro', lib=lib.dir)
#install.packages('reshape', lib=lib.dir)
#install.packages('scales', lib=lib.dir)
library(ggplot2, lib.loc = lib.dir) #"/Library/Frameworks/R.framework/Versions/3.2/Resources/library")
library(ggdendro, lib.loc = lib.dir) # "/Library/Frameworks/R.framework/Versions/3.2/Resources/library")
library(reshape, lib.loc = lib.dir) #"/Library/Frameworks/R.framework/Versions/3.2/Resources/library")
library(scales, lib.loc = lib.dir) #"/Library/Frameworks/R.framework/Versions/3.2/Resources/library")

### Installing Rlibstree and Bioconductor

To calculate the longest shared substring between two entries, we are using the `Rlibstree` package from Bioconductor. The following two cells install Bioconductor and the package we want, repectively, and only need to be run once. You will get a prompt box with no text. This is for updating the packages listed in the warning box. I suggest entering "n" for no, as the notebook seems to have a problem with updating all the listed packages and times out.

In [ ]:
#source("http://bioconductor.org/biocLite.R")
#biocLite("BiocInstaller")
#library(BiocInstaller)
#biocLite("Rlibstree")

library("Rlibstree")

In [ ]:
df = read.csv('../data/pass/ob_lists_wood_w_id_text.csv', stringsAsFactors = F) 

In [ ]:
df$lemma = as.character(df$lemma)
df$base = as.character(df$base)
df$entry = as.character(df$entry)
str(df)

In [ ]:
## Functions
get_guidewords = function(line) {
  # extract all guidewords for an entry into character vector
  line = unlist(strsplit(line, "_"))
  line = gsub(".*\\[", "", line)
  line = gsub("\\].*", "", line)
  line = gsub("~", "", line)
  guidewords = line
  guidewords = guidewords[which(guidewords != "na")]
  guidewords
}

get_citation_forms = function(line) {
  # extract all citation forms for an entry into character vector
  line = unlist(strsplit(line, "_"))
  line = gsub("(.*)\\[.*", "\\1", line)
  citation_form = line
  citation_form = citation_form[which(citation_form != "na")]
  citation_form
}

clean_kmer = function(x) {
  # get rid of part of speech (follows each ])
#  x = gsub("(\\])[a-zA-Z/]*_", paste0("\\1", "_"), x) #for all but last word
  x = gsub("(\\])[a-zA-Z/]*", "\\1", x) #for all but last word
#  x = gsub("(_.*\\])[a-zA-Z/]*", "\\1", x) #for last word
  
  # get rid of punctuation (but only {}[]_.)
  x = gsub("\\]", "", x)
  x = gsub("\\[", "", x)
  x = gsub("\\{", "", x)
  x = gsub("\\}", "", x)
  x = gsub("_", "", x)
  x = gsub("\\.", "", x)
  x = gsub("~", "", x)
  x
}

def_section_breaks = function(df, cutoff) {
  # a section ends anytime overlap is zero and k is below the defined cutoff
  df$section = NA
  sect_num = 1
  first_section_start = which(df$overlap > 0 | df$k >= cutoff)[1]
  df$section[first_section_start] = sect_num
  section_break = FALSE
  
  for (i in (first_section_start):nrow(df)) {
    if (df$overlap[i] > 0 | df$k[i] >= cutoff) {
      if (section_break == TRUE) { sect_num = sect_num + 1 }
      df$section[i] = sect_num
      section_break = FALSE
    } 
    else { 
      df$section[i] = NA
      section_break = TRUE
    }}
  df
}

extract_sections = function(df, file) {
  # writes all entries present in each section to a file
  section_nums = unique(df$section[!is.na(df$section)])
  sections = data.frame(sapply(section_nums, function(x) x = character(max(table(df$section)) + 1)))
  
  for (i in section_nums) {
    elements = unique(c(df[which(df$section == i),]$line_a, df[which(df$section == i),]$line_b))
    missing = max(table(df$section) + 1) - length(elements)
    elements = c(elements, rep(NA, missing))
    sections[,i] = elements 
    colnames(sections)[i] = paste(get_guidewords(sections[1,i]), collapse = "_")
  }
  write.csv(sections, file, row.names = FALSE, quote = FALSE)
}

In [ ]:
compare_entries = function(infile, cutoff, outfile) {
  name = strsplit(infile, "\\.")[[1]][1]
  df_composite = read.csv(infile, stringsAsFactors = FALSE)
  # remove lines in df representing missing lines or sections
  empty_lines = which(df_composite$entry == "")
  if (length(empty_lines) != 0) {df_composite = df_composite[-empty_lines,] }
  
  # initialize empty data frame for storing results
  num_rows = nrow(df_composite) - 1
  df_compare = data.frame(line_a = character(num_rows), 
                          line_b = character(num_rows),
                          overlap = numeric(num_rows),
                          kmer = character(num_rows),
                          k = numeric(num_rows),
                          stringsAsFactors = FALSE)
  
  for (i in 1:nrow(df_composite) - 1) {
    line_a = tolower(df_composite$entry[i])
    guidewords_a = get_guidewords(line_a)
    citation_form_a = get_citation_forms(line_a)
    
    line_b = tolower(df_composite$entry[i + 1])
    guidewords_b = get_guidewords(line_b)
    citation_form_b = get_citation_forms(line_b)
    
    line_a_clean = clean_kmer(line_a)
    line_b_clean = clean_kmer(line_b)
    kmer = getLongestCommonSubstring(c(line_a_clean, line_b_clean))
    kmer = gsub("[\x80-\xFF]", "", kmer) # get rid of multibyte strings introduced by Rlibstree
    k = nchar(kmer)
    if (length(kmer) == 0) {
        kmer = NA
        k = 0 }
    
    overlap = sum(citation_form_a %in% citation_form_b) + sum(guidewords_a %in% guidewords_b)
    
    df_compare$line_a[i] = line_a
    df_compare$line_b[i] = line_b
    df_compare$overlap[i] = overlap
    df_compare$kmer[i] = kmer
    df_compare$k[i] = k
  }
  plot(df_compare$overlap, pch = ".", main = name, ylab = "# matching words", xlab = "line number")
  df_compare = def_section_breaks(df_compare, cutoff = cutoff)
  plot(table(df_compare$section), main = name, ylab = "# entries in section", xlab = "section")
  extract_sections(df_compare, outfile)
  df_compare
}

In [ ]:
setwd("../data/composite_texts/")

Q1 = compare_entries("Q000001.csv", cutoff = 3, "Q000001_sections.csv")
Q39 = compare_entries("Q000039.csv", cutoff = 3, "Q000039_sections.csv")
Q40 = compare_entries("Q000040.csv", cutoff = 3, "Q000040_sections.csv")
Q41 = compare_entries("Q000041.csv", cutoff = 3, "Q000041_sections.csv")
Q42 = compare_entries("Q000042.csv", cutoff = 3, "Q000042_sections.csv")

In [ ]:
####### Read in section definitions from file #######

Q1_sections = read.csv("Q000001_sections.csv", stringsAsFactors = FALSE)
Q39_sections = read.csv("Q000039_sections.csv", stringsAsFactors = FALSE)
Q40_sections = read.csv("Q000040_sections.csv", stringsAsFactors = FALSE)
Q41_sections = read.csv("Q000040_sections.csv", stringsAsFactors = FALSE)
Q42_sections = read.csv("Q000042_sections.csv", stringsAsFactors = FALSE)

# Map sections to documents

For each document in a corpus, determine whether each section represented in the reference text is present in that document. For each section, determine how many times an entry from that section is present in each document.

In [ ]:
map_sections = function(corpus_file, section_defs) {
# Read in a csv listing all documents in corpus with text IDs
    corpus = read.csv(corpus_file)
# make df to store results
    doc_list = unique(corpus$id_text)
    section_names = colnames(section_defs)
    section_df = data.frame(sapply(doc_list, function(x) 
        x = numeric(length(section_names))), row.names = section_names)
    colnames(section_df) = doc_list
    doc_list = as.character(doc_list)
        
# iterate through sections and documents to populate df counts
    for (j in unique(doc_list)) {
        entries = as.character(corpus[which(corpus$id_text == j),]$entry)
        for (i in 1:ncol(section_defs)) {
            presence = sum(section_defs[,i] %in% tolower(entries))
            # if (presence > 0) presence = 1
            section_df[i, j] = presence 
            }
      section_df
    }
section_df
}


In [ ]:
test = map_sections("../pass/ob_lists_wood_w_id_text.csv", Q39_sections)
head(test)